In [1]:
import skmob
from skmob.utils import utils, constants
import pandas as pd
import geopandas as gpd
import numpy as np
from skmob.models import Gravity


##############################################################
#Paths
##############################################################

# INPUTS

# File downloaded from https://github.com/openpolis/geojson-italy/blob/master/geojson/limits_IT_provinces.geojson,
# The file contains information about the locations and ids of Italian provinces
in_Geojson_Provinces_IT = "input\\Italy_Provinces_Geojson.json"



# File downloaded from https://data.humdata.org/dataset/covid-19-mobility-italy. The populations for each province were 
# edited from https://www.citypopulation.de/en/italy/admin/
in_Italian_ProvincesID_Population_Path = "input\\Italy_ProvincesID_Population.csv"

# File downloaded from https://data.humdata.org/dataset/covid-19-mobility-italy. Contains origin-destination provinces
# from Italy, on a time scale
in_Origin_Destination_Matrix_Path = "input\\Italy_origin_destination_matrix.csv"

##############################################################

# OUTPUTS 

# The convered file from the geojson. Contains only the province id and the geometry of the province
out_ProvinceID_Geometry_CSV_Path = "output\\Italy_ProvinceID_Geometry.csv"

# Merged file which contains the Italian provinces along with population and ids
out_ProvinceID_Geometry_Population_Path = "output\\Italy_ProvinceID_Geometry_Population.csv"

# Path of the origin-destination Matrix created by Python
out_Origin_Dest_Flows_Path = "output\\Italy_Origin_Destination_Flows_Date.csv"
out_Origin_Dest_Flows_Mini_Path = "output\\Italy_Origin_Destination_Flows_Date_Mini.csv"
#############################################################

In [2]:
# Read geojson file containing provinces locations
geoJson_Path = open(in_Geojson_Provinces_IT)
gpd_df_geojson = gpd.read_file(geoJson_Path)

# Delete extra columns from geojson dataframe
del gpd_df_geojson['prov_istat_code_num']
del gpd_df_geojson['prov_acr']
del gpd_df_geojson['reg_istat_code_num']
del gpd_df_geojson['reg_istat_code']
del gpd_df_geojson['prov_name']
del gpd_df_geojson['reg_name']



# Read the provinces with population file
it_dtset_file = open(in_Italian_ProvincesID_Population_Path)
pd_df_ProvID_Population = pd.read_csv(it_dtset_file)

# Convert the geopandas to pandas dataframe, in order to merge it with the population dataframe
pd_Geojson = pd.DataFrame(gpd_df_geojson)

# Merge dataframes based on province id
pd_Geojson['prov_istat_code']=pd_Geojson['prov_istat_code'].astype(int)
df_Geometry_ProvID_Population = pd.merge(pd_Geojson,pd_df_ProvID_Population,on='prov_istat_code')
df_Geometry_ProvID_Population.columns = ['tile_ID', 'geometry', 'index', 'province_code', 'province_name', 'population']
df_Geometry_ProvID_Population.pop('index')
df_Geometry_ProvID_Population.pop('province_code')
tessellation = gpd.GeoDataFrame(df_Geometry_ProvID_Population, geometry=df_Geometry_ProvID_Population.geometry, crs = 'epsg:4326')

In [3]:
# Create limit the flows between 27/01-02/02 and 23/03-29/03

import datetime as DT
out_Origin_Dest_Flows_Path = "output\\Italy_Origin_Destination_Flows_Date.csv"
out_Origin_Dest_Flows_Path_January_1 = "output\\od_partial\\od_january_flows_20_26.csv"
out_Origin_Dest_Flows_Path_January_2 = "output\\od_partial\\od_january_flows_27_02.csv"
out_Origin_Dest_Flows_Path_March_1 = "output\\od_partial\\od_march_flows_30_05.csv"
out_Origin_Dest_Flows_Path_March_2 = "output\\od_partial\\od_march_flows_06_12.csv"

# create first dataset
temp_df = pd.read_csv(out_Origin_Dest_Flows_Path)
temp_df  = temp_df [temp_df['origin'] != temp_df['destination']]
startDate = DT.date(2020,1,20)
week_end = DT.date(2020,1,26)
temp_df['date'] = temp_df['date'].astype('datetime64[ns]')
temp_df['date'] = pd.to_datetime(temp_df['date']).dt.date
mask = (temp_df['date']>=startDate) & (temp_df['date']<=week_end)

# temp_df[mask].to_csv("hello.csv", index = False)
# print(temp_df[mask].head(100))
temp_df = temp_df[mask].groupby(['origin','destination']).sum().reset_index()
temp_df['flow'] = temp_df['flow'] 
temp_df.to_csv(out_Origin_Dest_Flows_Path_January_1, index = False)
# print(temp_df.head())

# create second dataset
temp_df = pd.read_csv(out_Origin_Dest_Flows_Path)
temp_df  = temp_df [temp_df['origin'] != temp_df['destination']]
startDate = DT.date(2020,1,27)
week_end = DT.date(2020,2,2)
temp_df['date'] = temp_df['date'].astype('datetime64[ns]')
temp_df['date'] = pd.to_datetime(temp_df['date']).dt.date
mask = (temp_df['date']>=startDate) & (temp_df['date']<=week_end)

# temp_df[mask].to_csv("hello.csv", index = False)
# print(temp_df[mask].head(100))
temp_df = temp_df[mask].groupby(['origin','destination']).sum().reset_index()
temp_df['flow'] = temp_df['flow'] 
temp_df.to_csv(out_Origin_Dest_Flows_Path_January_2, index = False)
# print(temp_df.head())

# create third dataset
temp_df = pd.read_csv(out_Origin_Dest_Flows_Path)
temp_df  = temp_df [temp_df['origin'] != temp_df['destination']]
startDate = DT.date(2020,3,30)
week_end = DT.date(2020,4,5)
temp_df['date'] = temp_df['date'].astype('datetime64[ns]')
temp_df['date'] = pd.to_datetime(temp_df['date']).dt.date
mask = (temp_df['date']>=startDate) & (temp_df['date']<=week_end)

# temp_df[mask].to_csv("hello.csv", index = False)
# print(temp_df[mask].head(100))
temp_df = temp_df[mask].groupby(['origin','destination']).sum().reset_index()
temp_df['flow'] = temp_df['flow'] 
temp_df.to_csv(out_Origin_Dest_Flows_Path_March_1, index = False)
# print(temp_df.head())


# create fourth dataset
temp_df = pd.read_csv(out_Origin_Dest_Flows_Path)
temp_df  = temp_df [temp_df['origin'] != temp_df['destination']]
startDate = DT.date(2020,4,6)
week_end = DT.date(2020,4,12)
temp_df['date'] = temp_df['date'].astype('datetime64[ns]')
temp_df['date'] = pd.to_datetime(temp_df['date']).dt.date
mask = (temp_df['date']>=startDate) & (temp_df['date']<=week_end)

# temp_df[mask].to_csv("hello.csv", index = False)
# print(temp_df[mask].head(100))
temp_df = temp_df[mask].groupby(['origin','destination']).sum().reset_index()
temp_df['flow'] = temp_df['flow'] 
temp_df.to_csv(out_Origin_Dest_Flows_Path_March_2, index = False)
# print(temp_df.head())

# Now we have created two files for january and march

In [4]:
# print(tessellation.head())
# FLOW DATAFRAME
fdf = skmob.FlowDataFrame.from_file(out_Origin_Dest_Flows_Path_January_1,
                                    tessellation=tessellation,
				                    tile_id='tile_ID',
                                    datetime = 'date',
                                    origin = 'origin',
                                    destination = 'destination',
                                    flow = 'flow',
				                    sep=",")

In [5]:
# print(fdf.head())
tot_outflows = fdf[fdf['origin'] != fdf['destination']].groupby(by='origin', axis=0)['flow'].sum().fillna(0).values
tessellation[constants.TOT_OUTFLOW] = tot_outflows
print(tessellation.head())

   tile_ID                                           geometry province_name  \
0        1  POLYGON ((7.89397 45.58222, 7.89654 45.57985, ...        Torino   
1        2  POLYGON ((7.92900 45.74244, 7.92584 45.74196, ...      Vercelli   
2        3  POLYGON ((8.42079 45.82981, 8.42028 45.83010, ...        Novara   
3        4  MULTIPOLYGON (((6.94540 44.42794, 6.94734 44.4...         Cuneo   
4        5  POLYGON ((7.96685 45.11667, 7.96729 45.11673, ...          Asti   

   population  tot_outflow  
0      430492       228554  
1      170911        81563  
2      208550       909623  
3      164788       460106  
4      214638       139784  


In [6]:
# Calculate synthetic model

# instantiate a singly constrained Gravity model
gravity_singly = Gravity(gravity_type='singly constrained')
print()
print(gravity_singly)

synth_fdf = gravity_singly.generate(tessellation,
                                   tile_id_column='tile_ID',
                                   tot_outflows_column='tot_outflow',
                                   relevance_column= 'population',
                                   out_format='flows')
print(synth_fdf.head())

synth_fdf.to_csv("synthetic_flows\\flow_unfitted.csv", index = False)




100%|██████████| 107/107 [00:00<00:00, 3690.77it/s]
Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")
  origin destination          flow
0      1           2   7962.632815
1      1           3   5392.615144
2      1           4   7355.436669
3      1           5  12505.372018
4      1           6  10116.478505

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


In [7]:
# fit the gravity model

gravity_singly_fitted = Gravity(gravity_type='singly constrained')
print()
print(gravity_singly_fitted)

gravity_singly_fitted.fit(fdf, relevance_column='population')
print(gravity_singly_fitted)

np.random.seed(0)
synth_fdf_fitted = gravity_singly_fitted.generate(tessellation,
                                                        tile_id_column='tile_ID',
                                                        tot_outflows_column='tot_outflow',
                                                        relevance_column= 'population',
                                                        out_format='flows')

print(synth_fdf_fitted.head())


Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")
100%|██████████| 107/107 [00:00<00:00, 2548.45it/s]Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.4577001170212007], origin_exp=1.0, destination_exp=0.25560467099834033, gravity_type="singly constrained")
  origin destination          flow
0      1           2  19575.218918
1      1           3   9990.502331
2      1           4  18399.051728
3      1           5  27309.938089
4      1           6  10918.201207
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)



In [8]:
synth_fdf_fitted.to_csv("synthetic_flows\\flow_fitted.csv", index = False)


In [10]:
import Levenshtein as Lv
df1 = pd.read_csv(out_Origin_Dest_Flows_Path_January_1)
df2 = pd.read_csv(out_Origin_Dest_Flows_Path_January_2)


LOD,NLOD = Lv.calculateLOD(df1,df2)
mNLOD = Lv.calculateMean(NLOD)
print("uncommon",mNLOD)

LOD,NLOD = Lv.calculateLOD(df1,df2)
mNLOD = Lv.calculateMean(NLOD)
print("common",mNLOD)

0.21801894552932036


In [11]:
df1 = pd.read_csv(out_Origin_Dest_Flows_Path_March_1)
df2 = pd.read_csv(out_Origin_Dest_Flows_Path_March_2)


LOD,NLOD = Lv.calculateLOD(df1,df2)
mNLOD = Lv.calculateMean(NLOD)
print(mNLOD)

0.6339750100008021


In [13]:
import os
fdf = skmob.FlowDataFrame.from_file(out_Origin_Dest_Flows_Path_January_1,
                                    tessellation=tessellation,
				                    tile_id='tile_ID',
                                    datetime = 'date',
                                    origin = 'origin',
                                    destination = 'destination',
                                    flow = 'flow',
				                    sep=",")
# print(fdf.head())
tot_outflows = fdf[fdf['origin'] != fdf['destination']].groupby(by='origin', axis=0)['flow'].sum().fillna(0).values
tessellation[constants.TOT_OUTFLOW] = tot_outflows
print(tessellation.head())

x_range = np.arange(-8,-1,.5).tolist()
x_range = [ round(elem, 2) for elem in x_range ]
y_range = [0.1,1.0]
z_range = [0.1,1.0]
# y_range = np.arange(0.1,1,0.5).tolist()
# y_range = [ round(elem, 2) for elem in y_range ]

directory = "synthetic_flows\\generated"
for f in os.listdir(directory):
    os.remove(os.path.join(directory, f))

for x in x_range:
    for y in y_range:
        for z in z_range:
            print(x,y,z)
            gravity_singly = Gravity(gravity_type='singly constrained', deterrence_func_type="power_law", deterrence_func_args=[x], origin_exp=y, destination_exp=z)
            print()
            print(gravity_singly)
            synth_fdf = gravity_singly.generate(tessellation,
                                            tile_id_column='tile_ID',
                                            tot_outflows_column='tot_outflow',
                                            relevance_column= 'population',
                                            out_format='flows')
            print(synth_fdf.head())

            synth_fdf.to_csv("synthetic_flows\\generated\\flow_singly_"+str(x)+"_"+str(y)+"_"+str(z)+".csv", index = False)        


100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 3822.78it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
  0%|                                                                          | 0/107 [00:00<?, ?it/s]

   tile_ID                                           geometry province_name  \
0        1  POLYGON ((7.89397 45.58222, 7.89654 45.57985, ...        Torino   
1        2  POLYGON ((7.92900 45.74244, 7.92584 45.74196, ...      Vercelli   
2        3  POLYGON ((8.42079 45.82981, 8.42028 45.83010, ...        Novara   
3        4  MULTIPOLYGON (((6.94540 44.42794, 6.94734 44.4...         Cuneo   
4        5  POLYGON ((7.96685 45.11667, 7.96729 45.11673, ...          Asti   

   population    tot_outflow  
0      430492   32650.571429  
1      170911   11651.857143  
2      208550  129946.142857  
3      164788   65729.428571  
4      214638   19969.142857  
-8.0 0.1 0.1

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-8.0], origin_exp=0.1, destination_exp=0.1, gravity_type="singly constrained")
  origin destination         flow
0      1           2  3819.336946
1      1           3   369.693851
2      1           4  3206.183281
3      1           5  95

100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 3963.85it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 3148.02it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


  origin destination          flow
0      1           2   3708.630172
1      1           3    429.401880
2      1           4   3012.686277
3      1           5  11391.041674
4      1           6    645.192592
-8.0 1.0 0.1

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-8.0], origin_exp=1.0, destination_exp=0.1, gravity_type="singly constrained")
  origin destination         flow
0      1           2  3819.336946
1      1           3   369.693851
2      1           4  3206.183281
3      1           5  9556.413862
4      1           6   295.011784
-8.0

100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 2816.59it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 3822.81it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


...

  origin destination         flow
0      1           2  1137.518974
1      1           3   770.373592
2      1           4  1050.776667
3      1           5  1786.481717
4      1           6  1445.211215
-1.5 0.1 0.1

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-1.5], origin_exp=0.1, destination_exp=0.1, gravity_type="singly constrained")
  origin destination         flow
0      1           2  1781.900138
1      1           3  1168.837935
2      1           4  1719.277861
3      1           5  2155.780395
4      1           6  1186.286667
-1.5

100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 2610.66it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
  0%|                                                                          | 0/107 [00:00<?, ?it/s]

 0.1 1.0

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-1.5], origin_exp=0.1, destination_exp=1.0, gravity_type="singly constrained")
  origin destination         flow
0      1           2   727.784593
1      1           3   571.044504
2      1           4   679.525217
3      1           5  1080.853796
4      1           6  1091.273479
-1.5 1.0 0.1

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-1.5], origin_exp=1.0, destination_exp=0.1, gravity_type="singly constrained")


100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 3101.65it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████████████████████████████████████████████████████████| 107/107 [00:00<00:00, 2734.46it/s]
C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


  origin destination         flow
0      1           2  1781.900138
1      1           3  1168.837935
2      1           4  1719.277861
3      1           5  2155.780395
4      1           6  1186.286667
-1.5 1.0 1.0

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-1.5], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")
  origin destination         flow
0      1           2   727.784593
1      1           3   571.044504
2      1           4   679.525217
3      1           5  1080.853796
4      1           6  1091.273479


In [15]:
import os
df1 = pd.read_csv(out_Origin_Dest_Flows_Path_January_1)
directory = "synthetic_flows\\generated"
txt = "filename,mNLOD\n"
for filename in os.listdir(directory):
    df2 = pd.read_csv(os.path.join(directory,filename))
    df2 = df2.astype(int)
    LOD,NLOD = Lv.calculateLOD(df1,df2)
    mNLOD = Lv.calculateMean(NLOD)
    txt += filename+","+str(mNLOD)+"\n"
    print(filename+","+str(mNLOD)+"\n")
print(txt)

flow_singly_-1.5_0.1_0.1.csv,0.7328978529183029

flow_singly_-1.5_0.1_1.0.csv,0.7516093215834345

flow_singly_-1.5_1.0_0.1.csv,0.7328978529183029

flow_singly_-1.5_1.0_1.0.csv,0.7516093215834345

flow_singly_-2.0_0.1_0.1.csv,0.6675261016909536

flow_singly_-2.0_0.1_1.0.csv,0.6913858880210655

flow_singly_-2.0_1.0_0.1.csv,0.6675261016909536

flow_singly_-2.0_1.0_1.0.csv,0.6913858880210655

flow_singly_-2.5_0.1_0.1.csv,0.6255818872667281

flow_singly_-2.5_0.1_1.0.csv,0.6471705540122283

flow_singly_-2.5_1.0_0.1.csv,0.6255818872667281

flow_singly_-2.5_1.0_1.0.csv,0.6471705540122283

flow_singly_-3.0_0.1_0.1.csv,0.6109197226623075

flow_singly_-3.0_0.1_1.0.csv,0.6219025743532551

flow_singly_-3.0_1.0_0.1.csv,0.6109197226623075

flow_singly_-3.0_1.0_1.0.csv,0.6219025743532551

flow_singly_-3.5_0.1_0.1.csv,0.6057171794031798



KeyboardInterrupt: 